In [4]:
import sys
sys.path.append('../ry')
from libry import *
from numpy import *
komo=0
lgp=0
K=None

In [7]:
if K is not None:
    K.clear()
else:
    K = Config()
K.addFile("../rai-robotModels/kuka_drake/setup.g")
K.addFile("../models/manip_station_contacts.g")
K.addFrame("obj0", "tabletop", "type:ssBox size:[.1 .1 .2 .02] color:[1. 0. 0.], contact, logical={ object }, joint:rigid, Q:<t(-0.2 0.1 .125)>" )
K.addFrame("obj1", "tabletop", "type:ssBox size:[.1 .1 .2 .02] color:[1. 0. 0.], contact, logical={ object }, joint:rigid, Q:<t(-0.2 -.3 .125)>" )
K.addFrame("obj2", "tabletop", "type:ssBox size:[.1 .1 .2 .02] color:[1. 0. 0.], contact, logical={ object }, joint:rigid, Q:<t(-0.2 -.1 .125)>" )
K.addFrame("obj3", "tabletop", "type:ssBox size:[.1 .1 .2 .02] color:[1. 0. 0.], contact, logical={ object }, joint:rigid, Q:<t(-0.2  .3 .125)>" )
V = K.view()
#K.setJointState([-0.1,-1,1.3],['iiwa_joint_2','iiwa_joint_4','iiwa_joint_6'])

In [8]:
coll = K.feature(FS.accumulatedCollisions, [])
K.computeCollisions()
coll.eval(K)
#K.getCollisions()
assert sum(x[2]<0 for x in K.getCollisions())<1

In [9]:
lgp = K.lgp("../models/fol-pickAndPlace.g")

#lgp.walkToDecision(0)

# This is also useful to check: inspect all decisions possible in this node, which expands the node.
# If there is no good decisions, the FOL rules are buggy
#lgp.getDecisions()

lgp.addTerminalRule("(held obj1)")
lgp.run(2)

#lgp.nodeInfo()
# this writes the initial state, which is important to check:
# do the grippers have the gripper predicate, do all objects have the object predicate, and tables the table predicate? These need to be set using a 'logical' attribute in the g-file
# the on predicate should automatically be generated based on the configuration

In [12]:
print(lgp.numSolutions())# wait until you have some number of solutions found (repeat executing this line...)
lgp.stop() #stops the thread... takes a while to finish the current job


1


In [13]:
# at a node, you can compute bounds, namely BT.seq (just key frames), BT.path (the full path),
# and BT.setPath (also the full path, but seeded with the BT.seq result)
lgp.optBound(BT.seq, True)
#lgp.nodeInfo()

In [14]:
komo=0

In [18]:
lgp.optBound(BT.path, True)
#lgp.nodeInfo()

In [19]:
# get the KOMO object for the seqPath computation of the 0. solution
komo = lgp.getKOMO(0, BT.seqPath)
#komo.displayTrajectory() #SOOO SLOOOW (TODO: add parameter for display speed)

In [20]:
# assign K to the 20. configuration of the 0. solution, check display
# you can now query anything (joint state, frame state, features)
i=0
x=0
configs = []
while True:
    try:
        configs.append(komo.getConfiguration(i))
    except:
        break
    i+=1

In [21]:
configs

[array([[  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           1.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           1.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           1.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           1.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00],
        [ -1.50000000e-02,   0.00000000e+00,   1.90000000e-01,
           1.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00],
        [  0.00000000e+00,   0.00000000e+00,   1.57500000e-01,
           7.07107000e-01,   0.00000000e+00,  -7.07107000e-01,
           0.00000000e+00],
        [  0.00000000e+00,   0.00000000e+00,   1.57500000e-01,
           7.